# HW 2 coding part

Due: SEP 29 AT 11:59PM

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Problem 1. KNN and Spam

## Background

The dataset consists of a collection of 57 features relating to about 4600 emails and a label of whether or not the email is considered spam. You have a training set containing about 70% of the data and a test set containing about 30% of the data. Your job is to build effective spam classification rules using the predictors.

## A note about features

The column names (in the first row of each .csv file) are fairly self-explanatory.

Some variables are named `word_freq_(word)`, which suggests a calculation of the frequency of how many times a specific word appears in the email, expressed as a percentage of total words in the email multiplied by $100$.

Some variables are named `char_freq_(*)`, which suggests a count of the frequency of the specific ensuing character, expressed as a percentage of total characters in the email multiplied by $100$.

Some variables are named `capital_run_length_(*)` which suggests some information about the average (or maximum length of, or total) consecutive capital letters in the email.

`spam`: This is the response variable, 0 = not spam, 1 = spam.

In [ ]:
dftr = pd.read_csv('spam_train_withlabels.csv',header=0) #(3721, 58)
dfte = pd.read_csv('spam_test_nolabels.csv',header=0) #(880, 57)

In [ ]:
vv = pd.concat([dftr.drop('spam',axis=1),dfte])

In [ ]:
np.sum(np.unique(vv.index,return_counts=True)[1]>1)

880

## Missing Values and outliers

This time look for missing values and outliers in the `capital_run_length_average` column

You can view missing values with (recall that an NA or `np.nan`) represent missing values. **DO NOT DELETE THEM**

ANS: There're 341

In [ ]:
print("Missing value in spam_train_withlabels.csv: ",dftr.isna().sum().sort_values(ascending=False)['capital_run_length_average'], ";",
      "Missing value in spam_train_nolabels.csv: ",dfte.isna().sum().sort_values(ascending=False)['capital_run_length_average'])

Missing value in spam_train_withlabels.csv:  341 ; Missing value in spam_train_nolabels.csv:  63


### Task 0
Output the total number of outlier values that you have clearly found. Explain your reasoning. **DO NOT DELETE THEM**

In [ ]:
# return binary column name
dftr.T[dftr.isin([0,1]).all()].T.columns

# exclude the binary column in the outlier value counting process
for column in dftr.columns[0:57]:
  outlier_num = 0
  col = dftr[column]
  outlier_num = ((col - col.mean()).abs() > 3 * col.std()).value_counts()[1]
  print(column, " has ", outlier_num, "outliers")

word_freq_make  has  68 outliers
word_freq_address  has  31 outliers
word_freq_all  has  77 outliers
word_freq_3d  has  12 outliers
word_freq_our  has  70 outliers
word_freq_over  has  80 outliers
word_freq_remove  has  89 outliers
word_freq_internet  has  55 outliers
word_freq_order  has  90 outliers
word_freq_mail  has  56 outliers
word_freq_receive  has  79 outliers
word_freq_will  has  84 outliers
word_freq_people  has  69 outliers
word_freq_report  has  80 outliers
word_freq_addresses  has  79 outliers
word_freq_free  has  60 outliers
word_freq_business  has  86 outliers
word_freq_email  has  86 outliers
word_freq_you  has  48 outliers
word_freq_credit  has  62 outliers
word_freq_your  has  67 outliers
word_freq_font  has  46 outliers
word_freq_000  has  87 outliers
word_freq_money  has  25 outliers
word_freq_hp  has  69 outliers
word_freq_hpl  has  85 outliers
word_freq_george  has  101 outliers
word_freq_650  has  90 outliers
word_freq_lab  has  50 outliers
word_freq_labs  has  

EXPLANATION: The total number of outlier values that I have found has been printed above. I first exclued the column with binary values and count outlier values among non-binary columns. Any column with z-score greater than 3 or less than -3 is considered to be an outlier. 

### Task 1

Split off the `spam` column into a new vector called `ytrain` and drop the column from the training dataframe. Use $k$-nearest neighbors regression **trained on the training dataset**--there are times when one could train on both test and train, but let's focus on train for now c.f. inductive learning--with $k = 15$ to impute the missing/outlier values in the `capital_run_length_average` column using the other predictors after standardizing (i.e. rescaling) them. You are allowed to just use the `sklearn` neighbors module to perform this. (Take a look at the pandas `concat` command and remember the `drop` command from the last HW.) Remember to rescale using information from the training set. Rescale the test set also.

When you are done with this part, you should have no more NA’s in the capital_run_length_average column in either the training or the test set. The mean (standard deviation) of the training dataset should be 0 (resp 1). The test should should be close to that, but not exactly that since you are rescaling based on the training data. There are times when it might make sense to scale based on the testing data (like if you assume the distribution has shifted a little bit), but we aren't doing that. Make sure you show all of your work.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

dlra = 'capital_run_length_average'
ytrain = dftr['spam']
xtrain = dftr.drop('spam', 1)
xtest = dfte

In [ ]:
clf = KNeighborsRegressor(15)
impute = xtrain.copy()
impute.dropna(inplace = True)
cap_run_avg = impute[dlra]
impute.drop(dlra, 1, inplace = True)
for name, data in impute.iteritems():
    impute[name] = (data-data.mean())/data.std()

clf.fit(impute, cap_run_avg)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                    weights='uniform')

In [ ]:
xtrain.drop(dlra, 1, inplace = True)
xtest.drop(dlra, 1, inplace = True)

xtrain[dlra] = clf.predict(xtrain)
xtest[dlra] = clf.predict(xtest)

xtrain = (xtrain - xtrain.mean())/xtrain.std()
xtest = (xtest - xtest.mean())/xtest.std()

impute_test_dlra = xtest[dlra]

In [ ]:
xtrain.mean(), xtrain.std()

(word_freq_make               -6.188725e-16
 word_freq_address            -2.968750e-16
 word_freq_all                -2.508668e-16
 word_freq_3d                 -3.963505e-16
 word_freq_our                 2.139290e-17
 word_freq_over                7.480057e-17
 word_freq_remove              1.207908e-15
 word_freq_internet            6.685506e-16
 word_freq_order              -3.535647e-17
 word_freq_mail                3.819991e-16
 word_freq_receive             5.794284e-16
 word_freq_will                3.429727e-16
 word_freq_people              3.469410e-16
 word_freq_report             -4.906792e-16
 word_freq_addresses          -1.645195e-16
 word_freq_free               -5.698508e-16
 word_freq_business           -4.699576e-16
 word_freq_email               4.946922e-16
 word_freq_you                -1.744223e-15
 word_freq_credit              6.187084e-16
 word_freq_your                1.429058e-15
 word_freq_font                2.697609e-16
 word_freq_000                 4

In [ ]:
xtest.mean(), xtest.std()

(word_freq_make               -4.107825e-16
 word_freq_address            -5.090625e-17
 word_freq_all                 4.030867e-16
 word_freq_3d                  9.323350e-17
 word_freq_our                -1.404180e-16
 word_freq_over               -6.926278e-17
 word_freq_remove              3.046805e-16
 word_freq_internet            1.261617e-17
 word_freq_order               4.061145e-16
 word_freq_mail                3.041759e-16
 word_freq_receive            -1.378947e-16
 word_freq_will                5.875351e-16
 word_freq_people              8.578996e-18
 word_freq_report              1.877286e-16
 word_freq_addresses          -1.108331e-16
 word_freq_free                5.588964e-17
 word_freq_business            2.985617e-16
 word_freq_email               2.527650e-16
 word_freq_you                 2.003826e-15
 word_freq_credit              1.468522e-16
 word_freq_your                1.680632e-16
 word_freq_font                2.025211e-16
 word_freq_000                 3

### Task 2

Write a function named `knnlearn()` that performs $k$-nearest neighbors classification, without resorting to a package. You do not  need to implement a fancy nearest neighbor search algorithm, just the naive search will suffice. Use the euclidean norm. Your function will implement some additional behavior

-  The function should automatically do a split of the training data into a sub-training set (80%) and a validation set (20%) for selecting the optimal $k$.
- The function should standardize each column: for a particular variable, say `x1`, compute the mean and standard deviation of `x1` **using the training set only**, say `mu1` and `s1`; then for each observed column in the training set and test set, subtract the respective `mu` and divide by the respective `s`. Your data should already be standardized, but now this code does it automatically.

In [ ]:
def euclid_dist(a, b):
  return np.sum((a-b)**2,axis=1)

In [ ]:
def knn_predict(x_train, x_test, y_train, num_neighbors):
  dist = []
  neigh_ind = []

  dists = [euclid_dist(i, x_train) for j, i in x_test.iterrows()]

  for row in dists:
    enum_neigh = enumerate(row)
    sorted_neigh = sorted(enum_neigh,
                          key=lambda x: x[1])[:num_neighbors]

    ind_list = [tup[0] for tup in sorted_neigh]
    dist_list = [tup[1] for tup in sorted_neigh]

    dist.append(dist_list)
    neigh_ind.append(ind_list)

  neighbors = np.array(neigh_ind)

  y_pred = np.array([np.argmax(np.bincount(y_train.take(neighbor))) for neighbor in neighbors])

  return y_pred

In [ ]:
def knn_eval_score(x_train, x_test, y_train, y_test, num_neighbors):
  y_pred = knn_predict(x_train, x_test, y_train, num_neighbors)

  return np.sum(y_pred == y_test) / len(y_pred)

In [ ]:
def knnlearn(xtrain,xtest,ytrain, max_k = 5):
    num_neighbors = range(2, max_k + 1)
    mask = np.random.rand(len(xtrain)) < 0.8
    xtrain_train = xtrain[mask]
    xtrain_test = xtrain[~mask]
    ytrain_train = ytrain[mask]
    ytrain_test = ytrain[~mask]
    """
    your code for producing a nearest neighbor estimate
    """
    for i in xtrain_train.columns:
      mu = np.mean(xtrain_train[i])
      s = np.std(xtrain_train[i])
      xtrain_train[i] = (xtrain_train[i] - mu) / s
      xtrain_test[i] = (xtrain_test[i] - mu) / s
    
    best_k = {'k': 0, 'score': 0}

    for i in num_neighbors:
      score = knn_eval_score(xtrain_train, xtrain_test, ytrain_train, ytrain_test, i)
      if score > best_k['score']:
        best_k['score'] = score
        best_k['k'] = i

    print(best_k)

    y_pred = knn_predict(xtrain_train, xtest, ytrain_train, best_k['k'])

    return y_pred


### Task 3

In this part, you will use your k-NN classifier to fit models on the actual dataset. If you weren’t able to successfully write a k-NN classifier in `Task 2`, you’re permitted to use `sklearn`. If you take this route, you may need to write some code to standardize the variables and select k, which `knnlearn()` from `Task 2` already does.

Now fit 2 models and produce 2 sets of predictions of spam on the test set:

1. `knnlearn()` using all predictors except for `capital_run_length_average` (say, if we were distrustful of our imputation approach). Call these predictions `knn_pred1`.

2. `knnlearn()` using all predictors including `capital_run_length_average` with the imputed values. Call these predictions `knn_pred2`.

Submit a `.csv` file called `assn2_NETID_results.csv` (to Canvas, **NOT** Gradescope) with the columns:

1. `capital_run_length_average`: the predictor in your test set that now contains the imputed values (so that we can check your work on imputation).
2. `knn_pred1`
3. `knn_pred2`

Make sure that row 1 here corresponds to row 1 of the test set, row 2 corresponds to row 2 of the test set, and so on.

In [ ]:
y_train_clean = dftr['spam']
x_train_clean = dftr.drop(['spam',dlra], 1)
x_test_clean = dfte.drop(dlra, 1)

knn_pred1 = knnlearn(x_train_clean, x_test_clean, y_train_clean, max_k = 30)
knn_pred2 = knnlearn(xtrain,xtest,ytrain, max_k = 30)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


{'k': 9, 'score': 0.8972972972972973}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
csv_df = pd.DataFrame({'Capital_Run_Length_Average':impute_test_dlra, 'knn_pred1': knn_pred1, 'knn_pred2': knn_pred2})
csv_df.to_csv('assn2_NETID_results.csv')

# Problem 2. Methods to solve Logistic Regression

We saw that the solution to least squares is $\hat{\beta} = (X^T X)^{-1} X^T y$. There is no such closed form solution for logistic regression. Recall that for logistic regression (when $y_i \in \{0,1\}$)

$$
\hat{\beta} = \arg \min_{\beta} f(\beta)
$$
where
$$
f(\beta) = \frac{1}{n} \sum_{i=1}^n y_i x_i^T \beta - \log(1+\exp(x_i^T \beta))
$$
Later on we will explore various methods to solve such problems. For now, one method is an interative method where we start with $\beta_0 = 0$ and update based on
$$
\beta_{k+1} = \beta_k - [H f(\beta_k)]^{-1} \nabla f(\beta_k)
$$
where
$$
[H f(\beta_k)]_{ij} = \frac{\partial^2 f(\beta_k)}{\partial_i \partial_j}
$$
is the matrix of second derivatives also known as the Hessian.

$$
\nabla f(\beta)=\frac{1}{n} \sum_{i=1}^{n}\left(-Y_{i} X_{i}^{T}+\frac{1}{1+e^{X_{i}^{T} \beta}} * e^{X_{i}^{T} \beta} * X_{i}^{T}\right)
$$

### Task 1
Write a function to implement 100 iterations of the above method.

In [ ]:
import numpy as np
def logreg(X,y):
    """
    input
    X: 2-d numpy array that is n by p
    y: numpy array of 0 and 1 of length n
    output
    B:
    2-d numpy array that is p by 100 where the $i^{th}$ column is beta_i.
    By definition the first (0^{th} in numpy) column of B should be all zeros.
    """
    n,p = X.shape ##python unpackaging a tuple
    assert n==y.shape[0]    
    B = np.zeros([p, 100])
    for j in range(1,100):
        fb = np.sum(np.dot(B[:, j - 1], X.T) * y - np.log(1 + np.exp(np.dot(B[:, j - 1], X.T)))) / len(X)
        print(fb)
        fb_grad = np.gradient(fb)
        hessian = np.empty((fb.ndim, fb.ndim) + fb.shape, dtype=fb.dtype) 
        for k, grad_k in enumerate(fb_grad):
            # iterate over dimensions
            # apply gradient again to every component of the first derivative.
            tmp_grad = np.gradient(grad_k) 
            for l, grad_kl in enumerate(tmp_grad):
                hessian[k, l, :, :] = grad_kl
        print(fb_grad)
        B[:,j] = B[:, j - 1] - hessian * fb_grad
    return B

### Task 2
Generate simulated data, so we can test the code. Take $p=5$ and $n=1000$. Plot the estimation error for each column of `B`. Compute the estimation error as $v_k = \| \beta_k - \beta^* \|_2^2$.

In [ ]:
#fill in code here
n=1000
p=5
betastar = np.ones(5)/np.sqrt(5)
X = np.random.randn(n,p)
y = np.zeros(n)
y[:round(n/2)] = 1
np.random.shuffle(y)

In [ ]:
Biterates = logreg(X,y)

In [ ]:
import matplotlib.pyplot as plt
v = np.sqrt(((Biterates.T - betastar) ** 2) / n)
plt.plot(np.log(v))

In [ ]:
v